In [2]:
!pip install pygogo

    100% |████████████████████████████████| 1.6MB 257kB/s 
  Running setup.py bdist_wheel for future ... - \ | / - done
  Stored in directory: /Users/echo030333/Library/Caches/pip/wheels/11/c5/d2/ad287de27d0f0d646f119dcffb921f4e63df128f28ab0a1bda
Successfully built future


In [37]:
import datetime
import json
import csv
from collections import OrderedDict
import time
import os
import re
import numpy as np
import sys

import urllib
from urllib.parse import urlencode
from urllib.request import urlopen
import requests

import psycopg2
import psycopg2.extras

import gensim

import sklearn
from sklearn import svm
from sklearn.externals import joblib

from elasticsearch import helpers, Elasticsearch
import ujson
import requests
from io import StringIO

import logging
import pygogo as gogo


In [22]:
!pip install pubnub

    100% |████████████████████████████████| 6.5MB 82kB/s 
  Running setup.py bdist_wheel for pubnub ... - \ | done
  Stored in directory: /Users/echo030333/Library/Caches/pip/wheels/fb/87/79/5af4a1a680d143563dec82391b37eb23d5ef5d8695d4cd4520
  Running setup.py bdist_wheel for pycryptodomex ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done
  Stored in directory: /Users/echo030333/Library/Caches/pip/wheels/76/28/aa/d08cf510d56c3f0be340ef6a5453e970ce14ca82c1c2a899d9
Successfully built pubnub pycryptodomex


In [23]:
from pubnub.callbacks import SubscribeCallback
from pubnub.enums import PNStatusCategory
from pubnub.pnconfiguration import PNConfiguration
from pubnub.pubnub import PubNub
 

In [90]:
def ProcessScrape():

   ##### Create logger #####
   log_format = '%(asctime)s - %(levelname)s - %(message)s'
   formatter = logging.Formatter(log_format)

   logger = gogo.Gogo(
       'status.fmt',
       low_hdlr=gogo.handlers.file_hdlr('ProcessScrape.log'),
       low_formatter=formatter,
       high_level='info',
       high_formatter=formatter).logger

   def log_message(message):
       logger.info(message)
      
   ### pubnub ###    
       pnconfig = PNConfiguration()
       pnconfig.subscribe_key = "sub-c-417d8a10-76d3-11e7-81e6-0619f8945a4f"
       pnconfig.publish_key = "pub-c-2882d102-7c5c-43e5-b6e1-096b075419fe"
       pnconfig.ssl = False
 
       pubnub = PubNub(pnconfig)
    
       class MySubscribeCallback(SubscribeCallback):
           def status(self, pubnub, status):
               pass
        # The status object returned is always related to subscribe but could contain
        # information about subscribe, heartbeat, or errors
        # use the operationType to switch on different options
               if status.operation == PNOperationType.PNSubscribeOperation \
                    or status.operation == PNOperationType.PNUnsubscribeOperation:
                    if status.category == PNStatusCategory.PNConnectedCategory:
                        pass
                # This is expected for a subscribe, this means there is no error or issue whatsoever
                    elif status.category == PNStatusCategory.PNReconnectedCategory:
                        pass
                # This usually occurs if subscribe temporarily fails but reconnects. This means
                # there was an error but there is no longer any issue
                    elif status.category == PNStatusCategory.PNDisconnectedCategory:
                        pass
                # This is the expected category for an unsubscribe. This means there
                # was no error in unsubscribing from everything
                    elif status.category == PNStatusCategory.PNUnexpectedDisconnectCategory:
                        pass
                # This is usually an issue with the internet connection, this is an error, handle
                # appropriately retry will be called automatically
                    elif status.category == PNStatusCategory.PNAccessDeniedCategory:
                        pass
                # This means that PAM does allow this client to subscribe to this
                # channel and channel group configuration. This is another explicit error
                    else:
                        pass
                # This is usually an issue with the internet connection, this is an error, handle appropriately
                # retry will be called automatically
               elif status.operation == PNOperationType.PNSubscribeOperation:
            # Heartbeat operations can in fact have errors, so it is important to check first for an error.
            # For more information on how to configure heartbeat notifications through the status
            # PNObjectEventListener callback, consult <link to the PNCONFIGURATION heartbeart config>
                 if status.is_error():
                    pass
                # There was an error with the heartbeat operation, handle here
                 else:
                    pass
                # Heartbeat operation was successful
               else:
                pass
            # Encountered unknown status type
 
           def presence(self, pubnub, presence):
               pass  # handle incoming presence data
 
           def message(self, pubnub, message):
               pass  # handle incoming messages
 
 

           def publish_callback(result, status):
               pass
    
    # Handle PNPublishResult and PNStatus
       pubnub.subscribe().channels('test').execute()   
       pubnub.publish().channel('test').message(['logging', 'client', message() ]).async(publish_callback)


   #####  Getting and Processing Data #####
   log_message("Getting and Processing Data")
   _,_,files = next(os.walk("./JSONs/"))
   log_message("Number of files: "+str(len(files)))

   data = []
   for filename in files:    
       with open('./JSONs/'+filename, encoding="utf8") as json_file:
           json_data = json.load(json_file, object_pairs_hook=OrderedDict)
           for listing in json_data:
               data.append(listing)

   for filename in files:
       os.remove('./JSONs/'+filename)

   #strips all non-ascii characters
   #this is for all remaining non-significant ones
   def make_ascii(text):
       return ''.join([i if ord(i) < 128 else '' for i in text])

   #strips out all html markup and insert ascii replacements
   def cleanhtml(raw_html):
       #replace for \n
       raw_html = raw_html.replace("<br>","\n")
       raw_html = raw_html.replace("</li>","\n")
       raw_html = re.sub("</h*>","\n", raw_html)
       raw_html = re.sub("</h3>","\n", raw_html)
       #html conversions
       raw_html = raw_html.replace("&nbsp;"," ")
       raw_html = raw_html.replace("   "," ")
       raw_html = raw_html.replace("&#x27;","'")
       raw_html = raw_html.replace("â","'")    
       raw_html = raw_html.replace("â","-")
       raw_html = raw_html.replace("â","-")
       raw_html = raw_html.replace("â¢","-")
       raw_html = raw_html.replace("â","-")
       raw_html = raw_html.replace("&lt;","<")
       raw_html = raw_html.replace("&gt;",">")
       raw_html = raw_html.replace("â¦","...")
       raw_html = raw_html.replace("â¦","...")
       raw_html = raw_html.replace("&amp","and")
       raw_html = raw_html.replace("and;","and")
       Raw_html = raw_html.replace("&","and")
       raw_html = raw_html.replace("Ã©","e")
       raw_html = raw_html.replace("â¬","€")
       raw_html = raw_html.replace("â","\"") 
       raw_html = raw_html.replace("â","\"")       
       raw_html = raw_html.replace("Â","")
       raw_html = raw_html.replace("__","")
       raw_html = raw_html.replace("_","")
       raw_html = raw_html.replace("ï¿½","")
       raw_html = re.sub("##+","",raw_html)
       #remove noise markups
       cleantext = raw_html
       cleantext = re.sub('<.*?>','',cleantext)
       cleantext = re.sub(' +',' ',cleantext)
       cleantext = re.sub('(\n)(\n)(\n)+','\n\n',cleantext)
       return make_ascii(cleantext)

   #iterate through listing descriptions and clean each one
   for job_number, job in enumerate(data):   
       job["description"] = cleanhtml(str(job["description"]))
       job["label"] = 0

   raw_data = data

   #####  OpenCalais #####
   log_message("Using OpenCalais")
   # this is slow
   # set API key and url.
   access_token = 'AiYCbxhaNTAxGKBe5cl6KFWCM03GEEy3'
   calais_url = 'https://api.thomsonreuters.com/permid/calais'

   for i, row in enumerate(raw_data):
       input_data = row['description']
       
       # set headers for Calais.
       headers = {'X-AG-Access-Token' : access_token, 'Content-Type' : 'text/raw', 'outputformat' : 'application/json'}    
              
       if(i%20==0):
          log_message(i+1,'/',len(raw_data))
       resp_json = "nothing here"
       for i in range(10):
           try:
               response = requests.post(calais_url, data=input_data,headers=headers)
               resp_json = response.json()
               break
           except:
               continue
       if(resp_json=="nothing here"):
           continue
                   
       new_tags = []
       keys = resp_json.keys()
       for key in keys:
           field = resp_json[key]
           try:
               typeGroup = field['_typeGroup']
               if(typeGroup=="socialTag"):
                   new_tags.append(field['name'])
           except:
               continue

       new_tags = list(set(new_tags))
       tags = row['tags']
       for tag in new_tags:
           tags.append(tag)
       row['tags'] = tags

   ##### Formatting and Processing Data #####
   log_message("Formatting and Processing Data")

   #split features based on symbol
   def resplit(arr,symbol):
       retarr = []
       for value in arr:
           vals = value.split(symbol)
           for val in vals:
               retarr.append(val)
       return retarr

   #load in full data
   full_data = {}

   for i, row in enumerate(raw_data):
       myID = row['id']
       tags = row['tags']
       if(tags==None):
           tags="[]"
       position = row['position']
       features = [] 
               
       #split tags and position based on any symbols
       splts = [" ","-","/"]
       for splt in splts:
           tags = resplit(tags,splt)
       for tag in tags:
           features.append(tag)
                
       splts = ["-","/"]
       position = position.split(" ")
       for splt in splts:
           position = resplit(position,splt)
       for pos in position:
           features.append(pos)
    
       full_data[myID] = features 
            
   for myID in full_data:
       features = full_data[myID]
       for num_feat, feature in enumerate(features): 
           #strip any excess symbols
           strip_items = ["(",")","-","+","/","&"]
           for strip_item in strip_items:
               feature = feature.strip(strip_item)
               features[num_feat] = feature.strip(strip_item)                        
       while('' in features):
           features.remove('')

   #reformatted data
   label_data = []
   for myID in full_data:
       params = [myID, full_data[myID]]
       label_data.append(params)

   ##### Word2Vec and Preprocessing #####
   log_message("Word2Vec and Preprocessing")

   #loads in pretrained word2vec model
   #to get vector, run as dictionary
   model = gensim.models.KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True) 

   X = []
   for row in label_data:
       words = row[1]
       words_vec = []
       for word in words:
           try:
               words_vec.append(model[word])
           except:
               continue
       X.append(words_vec)

   ##### Classifier #####
   log_message("Classifier")

   #loads saved model
   acc_name="36.32"
   clf = joblib.load('./models/'+'svm_'+acc_name+'.pkl')

   y = np.zeros((len(X)))
   for user_index, user in enumerate(X):
       cats = np.zeros((7))
       try:
           user_pred = clf.predict(user)
       except: #result is misc
           cats[6]+=1
           continue
       for pred in user_pred:
           for i in range(7):
               if( abs(pred-(i+1)) <.1):
                   cats[i]+=1       
       for i in range(7):
           cats[i] += i/10
       y[user_index] = np.argmax(cats)+1

   hashed_labels = {}
   for i, row in enumerate(label_data):
       hashed_labels[row[0]] = int(y[i])

   for listing in raw_data:
       listing['label'] = hashed_labels[listing['id']]

   ##### Data Changes #####
   log_message("Data Changes")

   for listing in raw_data:
       checks = ['company','description','epoch','position','slug']
       for check in checks:
           if(listing[check]==None):
               listing[check] = "No "+check+" provided"

   #add timepast
   today = datetime.datetime.today()
   for listing in raw_data:
       date = listing['date']
       if(date==None):
           listing['timepast'] = 100
           continue
       date = datetime.datetime.strptime(date[0:10], '%Y-%m-%d')
       date = date.replace(tzinfo=None)
       difference = today - date
       listing['timepast'] = difference.days

   #format dates
   for listing in raw_data:
       date = listing['date']
       if(date==None):
           listing['date'] = "No date provided"
           continue
       date = datetime.datetime.strptime(date[0:10], '%Y-%m-%d')
       disp_date = date.strftime('%b %d, %Y')
       listing['date'] = disp_date

   #format tags
   for listing in raw_data:
       tags = listing['tags']
       tags_disp = "No tags"
       if(len(tags)>0):
           tags_disp = ""
           for tag in tags:
               tag = tag.title()
               tags_disp += tag+", "
           tags_disp = tags_disp[0:len(tags_disp)-2]
       tags_arr = tags_disp.split(", ")
       listing['tags'] = tags_arr
       listing['tags_display'] = tags_disp

   #Uses first paragraph as description
   for listing in raw_data:
       description = listing['description']
       if(description==None):
           description = "No description provided."
       else:
           description = description.split("\n")[0]
       listing['description'] = description

   #Gives dummy logos to those without
   for listing in raw_data:
       logo = listing['logo']
       if(logo==None or len(logo)<3):
           logo = "https://dummyimage.com/170/000/ffffff&text="+(listing['company'].replace(' ','%20'))
       listing['logo'] = logo

   # Converts labels to categories
   ## Labels: 1=Interface Tech Arch(ITA), 2=Software Tech Arch(STA), 3=Database Tech Arch(DTA), 4=System Tech Arch(SysTA),
   # 5=Bus Anal (BA), 6=Proj Man (PM), 7=N/A
   label_to_name = {
       "0": "Uncategorized",
       "1": "Interface Technical Architect",
       "2": "Software Technical Architect",
       "3": "Database Technical Architect",
       "4": "System Technical Architect",
       "5": "Business Analyst",
       "6": "Project Manager",
       "7": "Miscellaneous"
   }
   for listing in raw_data:
       listing['label'] = label_to_name[str(listing['label'])]

   for listing in raw_data:
       listing['title'] = "title"

   for listing in raw_data:    
       if(listing['timepast']==0):
           listing['new'] = " [New!]"
       else:
           listing['new'] = ""

   ##### JSON Generation #####
   log_message("JSON Generation")
   file_name = "mindy_els"
   with open(file_name+".json", 'w', encoding="utf8") as outfile:
       json.dump(raw_data, outfile)

   ##### ElasticSearch #####
   log_message("ElasticSearch")
   es = Elasticsearch()
   with open(file_name+".json", 'r', encoding="utf8") as outfile:
       data = json.load(outfile)    
       helpers.bulk(es, data, index='my_index', doc_type='my_type')

   #gets duplicates
   def fetch_duplicates():
       uri = "http://mindy-elastic:9200/my_index/_search"
       payload = {"size": 0,
                   "aggs": {
                       "duplicateCount": {"terms":
                               {"field": "id", "min_doc_count": 2, "size": 10000},
                                   "aggs": {
                                       "duplicateDocuments":
                                       {"top_hits": {}}
                                   }
                               }
                       }
                  }
       json = ujson.dumps(payload)
       resp = requests.post(uri, data=json)
       ret = ujson.loads(resp.text)
       return ret

   # generates commands to delete
   def delete_query(buf, index, doc_type, i):
       buf.write('{"delete":{"_index":"'+index+'","_type":"'+doc_type+'","_id":"'+i+'"}}\n')

   # bulk deletes and counts
   def bulk_remove(buf):
       try:
           uri = "http://mindy-elastic:9200/_bulk?refresh=wait_for"
           resp = requests.post(uri, data=buf.getvalue())
           if resp.status_code == 200:
               r = ujson.loads(resp.text)
               if r['errors']:
                   log_message(r)
               cnt = 0
               for item in r['items']:
                   if ('found' in item['delete']) and item['delete']['found']:
                       cnt += 1
                   else:
                       log_message(item)
               return cnt
           elif(resp.text == '''{"error":{"root_cause":[{"type":"parse_exception","reason":"request body is required"}],"type":"parse_exception","reason":"request body is required"},"status":400}'''):
               log_message("No more to remove")
           else:
               log_message("failed to fetch duplicates: #{0}".format(resp.text))
       except requests.exceptions.ConnectionError as e:
           logger.error("ERROR: connection failed, check --host argument and port. Is ES running on http://mindy-elastic:9200?")
           logger.error(e)

   # keeps calling the remove process until all are removed
   # this is necessary because of how elastic shards data
   # sometimes duplicates are not caught due to this and requires loop
   duplicatesExist = 2 #check twice
   timer = 0 #prevent infinite loop
   while(duplicatesExist>0 and timer<10):
       resp = fetch_duplicates()
       docs = len(resp["aggregations"]["duplicateCount"]["buckets"])    
       if(docs==0):
           duplicatesExist-=1
       removed = remove_duplicates(resp, "my_index", "my_type")
       log_message(docs," ",removed)
       timer+=1
    
